#### Restart & Run All | Send orders.csv to obsidian

In [1]:
import calendar
import pandas as pd
import sidetable
import panel as pn
import seaborn as sns
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=False)
pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

today = date.today()
today

<IPython.core.display.Javascript object>

datetime.date(2023, 2, 6)

### Begin of Tables in the process

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name qty target active spd current change percent reason market xdate'.split()

format_dict = {
    'qty':'{:,}',
    'price':'{:.2f}','target':'{:.2f}','current':'{:.2f}','change':'{:.2f}','diff':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}'
}

pd.read_sql_query('SELECT * FROM orders ORDER BY id DESC LIMIT 1', conlite).style.format(format_dict)

,id,trade,name,qty,price,active,reason,market,xdate
0,21,B,CPNREIT,"10,000",18.30,2,HD,PF&REIT,2022-02-02


### Print to verify before upload file

In [3]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)

df_tab = pn.widgets.Tabulator(orders, layout='fit_data', width=710)
df_tab

Tabulator(layout='fit_data', value=   trade     n..., width=710)

In [4]:
orders.stb.freq(["market"])

,market,count,percent,cumulative_count,cumulative_percent
0,SET50,8,38.095238,8,38.095238
1,SET100,8,38.095238,16,76.190476
2,SET,4,19.047619,20,95.238095
3,PF&REIT,1,4.761905,21,100.000000


In [5]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)
orders[cols].to_csv(osd_file, header=True, index=False)

### End of transactional process

### Process to set target price

#### 1) Set50 records

In [6]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET50" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,BANPU,6000,10.80,0,DTD,SET50,2023-04-08
1,6,B,COM7,3000,30.00,0,XXX,SET50,2022-12-22
2,7,B,CPF,4000,23.00,0,XXX,SET50,2022-12-22
3,9,B,PTT,3600,32.00,0,XXX,SET50,2023-03-03
4,10,B,TTB,90000,1.40,0,52WL,SET50,2022-12-22
5,17,S,IVL,2400,44.25,0,DTD,SET50,2023-05-06
6,18,S,JMART,3600,41.50,0,DTD,SET50,2023-04-19
7,19,S,JMT,1500,60.00,0,DTD,SET50,2023-04-19


In [7]:
name = 'JMART'
limit = 20 # 1 month of data
sql = """
SELECT P.*
FROM price P
WHERE P.name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe().round(2)

,price,maxp,minp,qty,opnp
count,20.00,20.00,20.00,20.00,20.00
mean,39.42,40.26,38.99,10298126.50,39.72
std,2.25,2.13,2.23,7417622.97,2.18
min,35.50,37.50,35.25,4039787.00,36.00
25%,37.44,38.00,37.19,5672148.75,37.75
50%,39.00,39.88,38.75,8022339.50,39.75
75%,41.56,42.31,41.12,10738002.50,41.75
max,42.50,43.50,42.00,35425033.00,42.50


#### 2) Set100 records

In [8]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET100" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,4,B,CK,4500,22.50,0,5pct,SET100,2022-12-22
1,5,B,CKP,20000,4.50,0,3L,SET100,2022-12-22
2,2,B,KCE,1000,54.25,0,CUT,SET100,2023-03-21
3,8,B,MEGA,2000,48.00,0,5pct,SET100,2023-03-08
4,3,B,RCL,3000,28.75,0,HD,SET100,2023-03-23
5,16,S,ORI,15000,12.50,0,CP1S,SET100,2023-05-09
6,20,S,SINGER,1200,29.25,0,DTD,SET100,2023-04-29
7,14,S,STA,2500,23.00,0,CUT,SET100,2023-04-19


In [9]:
name = 'SINGER'
limit = 60 # 3 month of data
sql = """
SELECT P.*
FROM price P
WHERE P.name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe().round(2)

,price,maxp,minp,qty,opnp
count,60.00,60.00,60.00,60.00,60.00
mean,30.07,30.72,29.59,5826349.97,30.24
std,2.35,2.46,2.36,3305977.03,2.44
min,26.75,27.25,26.25,1984596.00,27.00
25%,28.25,28.75,27.69,3165197.75,28.25
50%,29.38,30.00,29.12,5087148.50,29.62
75%,31.75,32.31,31.25,7381591.75,32.00
max,36.50,37.50,36.00,15522537.00,37.00


#### 3) Set records

In [10]:
# Why SIS does not print out from this SQL?
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,12,S,DIF,10000,14.0,0,CUT,SET,2023-02-10
1,11,S,JASIF,10000,8.5,0,CP1S,SET,2023-03-03
2,15,S,TMT,12000,8.7,0,C9.0,SET,2023-04-18
3,13,S,WHAIR,10000,8.9,0,HD,SET,2023-03-04


In [11]:
name = 'TMT'
limit = 120 # 6 month of data
sql = """
SELECT P.*
FROM price P
WHERE P.name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe().round(2)

,price,maxp,minp,qty,opnp
count,120.00,120.00,120.00,120.00,120.00
mean,7.75,7.81,7.69,361217.69,7.76
std,0.40,0.39,0.39,310838.91,0.40
min,6.95,7.05,6.95,54095.00,7.00
25%,7.55,7.60,7.50,179582.75,7.59
50%,7.85,7.95,7.80,304466.50,7.85
75%,8.00,8.05,7.95,422134.00,8.00
max,8.45,8.50,8.30,2565557.00,8.45


In [12]:
pd.read_sql_query('SELECT * FROM orders ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,BANPU,6000,10.80,0,DTD,SET50,2023-04-08
1,4,B,CK,4500,22.50,0,5pct,SET100,2022-12-22
2,5,B,CKP,20000,4.50,0,3L,SET100,2022-12-22
3,6,B,COM7,3000,30.00,0,XXX,SET50,2022-12-22
4,7,B,CPF,4000,23.00,0,XXX,SET50,2022-12-22
5,21,B,CPNREIT,10000,18.30,2,HD,PF&REIT,2022-02-02
6,2,B,KCE,1000,54.25,0,CUT,SET100,2023-03-21
7,8,B,MEGA,2000,48.00,0,5pct,SET100,2023-03-08
8,9,B,PTT,3600,32.00,0,XXX,SET50,2023-03-03
9,3,B,RCL,3000,28.75,0,HD,SET100,2023-03-23
